#Taller Clustering en letras de canciones 
###Sergio Lopez Martinez - PLN - 2019-l

##Importar los conjuntos de datos que utilizaremos

Tenemos 2 conjuntos, el primero contiene las letras originales, el segundo  de letras luego de ser procesadas con PartOfSpeech utilizando Freeling: http://nlp.lsi.upc.edu/freeling/demo/demo.php

In [1]:
!wget -O Lyrics.zip "https://drive.google.com/uc?export=download&id=1sB0VJnqBR5PbpNLfW73UeZFIbwlL3cv4"
!unzip Lyrics.zip

--2019-07-12 02:43:47--  https://drive.google.com/uc?export=download&id=1sB0VJnqBR5PbpNLfW73UeZFIbwlL3cv4
Resolving drive.google.com (drive.google.com)... 74.125.141.100, 74.125.141.101, 74.125.141.139, ...
Connecting to drive.google.com (drive.google.com)|74.125.141.100|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-08-40-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/nfogtup9pafgnc3506berhan8ie0u2m9/1562896800000/10519510251368043159/*/1sB0VJnqBR5PbpNLfW73UeZFIbwlL3cv4?e=download [following]
--2019-07-12 02:43:48--  https://doc-08-40-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/nfogtup9pafgnc3506berhan8ie0u2m9/1562896800000/10519510251368043159/*/1sB0VJnqBR5PbpNLfW73UeZFIbwlL3cv4?e=download
Resolving doc-08-40-docs.googleusercontent.com (doc-08-40-docs.googleusercontent.com)... 74.125.141.132, 2607:f8b0:400c:c06::84
Connecting to doc-08-40-docs.googleusercontent.com (doc-08

In [2]:
!wget -O POS-Freeling.zip "https://drive.google.com/uc?export=download&id=1hv4kKcpgZRy79L9PKESrK6j6MClY-FIw"
!unzip POS-Freeling.zip

--2019-07-12 02:43:52--  https://drive.google.com/uc?export=download&id=1hv4kKcpgZRy79L9PKESrK6j6MClY-FIw
Resolving drive.google.com (drive.google.com)... 74.125.141.139, 74.125.141.138, 74.125.141.100, ...
Connecting to drive.google.com (drive.google.com)|74.125.141.139|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0k-40-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/0e0dcrdj25hfngeu4froqs4pfrubr87t/1562896800000/10519510251368043159/*/1hv4kKcpgZRy79L9PKESrK6j6MClY-FIw?e=download [following]
--2019-07-12 02:43:53--  https://doc-0k-40-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/0e0dcrdj25hfngeu4froqs4pfrubr87t/1562896800000/10519510251368043159/*/1hv4kKcpgZRy79L9PKESrK6j6MClY-FIw?e=download
Resolving doc-0k-40-docs.googleusercontent.com (doc-0k-40-docs.googleusercontent.com)... 74.125.141.132, 2607:f8b0:400c:c06::84
Connecting to doc-0k-40-docs.googleusercontent.com (doc-0k

##Preprocesamiento

Creamos un directorio para relizar la lematizacion de las letras originales

In [0]:
!mkdir Lemma

In [4]:
import json
import glob

#Import files
files = (glob.glob("Lyrics/*"))

#Initialize banned words.
banned_words = {
    "yeah": 1,
    "oh": 1
}

#Clean all words from all files
for file in files:
    filename = file[file.rfind("/") + 1:]
    with open(file, 'r') as f:
        write = ""
        for line in f:
            words = map(str, line.strip().split(' '))
            for w in words:
                if w not in banned_words.keys():
                    if len(w) > 2:
                        write += w.lower() + " "
        write += '\n'
    with open("Lemma/"+ filename, 'w') as l:
        l.write(write)
print ("\n\nLemma done succesfully.")




Lemma done succesfully.


###Ahora con el siguiente script sobre el conjunto de datos preprocesado de PartOfSpeech, crea las palabras procesadas con las frecuencias que cada una tiene en su archivo de letras

In [0]:
!mkdir DataJson

In [6]:

#Import files
files = (glob.glob("POS-Freeling/*"))
c = 0
#Import Lemma files from Freeling Software
for file in files:
    d = {}
    filename = file[file.rfind("/") + 1:]
    filename_small = filename.replace(".txt", ".json")
    with open(file, 'r') as f:
        for line in f:
            line = line.strip()
            if(c % 3 == 1):
                if line in d:
                    d[line] += 1
                else:
                    d[line] = 1
            c += 1
    #Export data to json
    with open("DataJson/" + filename_small, 'w') as fp:
        json.dump(d, fp, sort_keys=True, indent=4)

print ("\n\nDataJson done succesfully")




DataJson done succesfully


In [0]:
!mkdir Cleaned

## Removiendo stopwords
Este archivo limpia las palabras que ya se han procesado en la Extracción de la parte de la letra de la parte de la letra (POS), eliminando todas las palabras clave encontradas en cada archivo de letras, utilizando una lista de palabras clave estándar que se encuentra en el paquete NLTK

In [8]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
import json
from os import listdir
from os.path import isfile, join

# Getting stopwords from nltk
stop_words = set( stopwords.words('spanish') )

# File path of processed words
filepath = 'DataJson/'

# Getting names of all the files in the lemma folder
onlyfiles = [f for f in listdir(filepath) if isfile(join(filepath, f))]

# Looping all files in folder
for filename in onlyfiles:

    # Get the words of the file
    data = json.load( open(filepath + filename) )
    word_tokens = data.keys()

    # Cutting the stopwords from the words array
    filtered_sentence = []
    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)

    # Creating a new file to save the new json with the original frequencies
    new_dict = {}
    for key in filtered_sentence:
        new_dict[key] = data[key]

    # Saving the file
    with open('Cleaned/' + filename, 'w') as outfile:
        json.dump(new_dict, outfile)

print ("Added all cleaned files to files/Cleaned")

Added all cleaned files to files/Cleaned


##Calculo de tf-idf

Utilizando el conjunto anterior y el vectorizador tf-idf proporcionado por la libreria sklearn, podremos hacerlo de una manera sencilla

In [10]:
#Import files
files = (glob.glob("Cleaned/*"))
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(files)
vectors.shape

(16, 60)

Nuestra matriz se encuentra en un formato de fila dispersa, debido a su composicion, recordemos que esta matiz contiene bastantes 0s por lo cual su espacio en memoria por sus dimensiones es elevado, por lo cual python utiliza este formato para disminuir dicho espacio de almacenamiento.

In [11]:
vectors

<16x60 sparse matrix of type '<class 'numpy.float64'>'
	with 95 stored elements in Compressed Sparse Row format>

Sin embargo, con el atributo data nos permite ver los valores que no son 0 dentro de la matriz

In [12]:
vectors.data

array([0.12070278, 0.3790147 , 0.3790147 , 0.3790147 , 0.3790147 ,
       0.3790147 , 0.3790147 , 0.33007394, 0.12070278, 0.40086564,
       0.40086564, 0.46030284, 0.46030284, 0.46030284, 0.14659017,
       0.14659017, 0.40151773, 0.40151773, 0.40151773, 0.40151773,
       0.40151773, 0.40151773, 0.1278692 , 0.1278692 , 0.55876914,
       0.55876914, 0.55876914, 0.17794821, 0.17794821, 0.58111448,
       0.58111448, 0.50607732, 0.18506441, 0.18506441, 0.60637457,
       0.52807567, 0.52807567, 0.19310885, 0.19310885, 0.46030284,
       0.46030284, 0.46030284, 0.40086564, 0.14659017, 0.40086564,
       0.14659017, 0.55876914, 0.55876914, 0.55876914, 0.17794821,
       0.17794821, 0.6737652 , 0.6737652 , 0.21457038, 0.21457038,
       0.48778518, 0.48778518, 0.48778518, 0.48778518, 0.15534233,
       0.15534233, 0.50243895, 0.50243895, 0.50243895, 0.43756087,
       0.16000903, 0.16000903, 0.55876914, 0.55876914, 0.55876914,
       0.17794821, 0.17794821, 0.55876914, 0.55876914, 0.55876

##Clustering

Para facilidad en el clustering se utilizo CLUTO por su facilidad para evaluar el modelo en bisecciones repetidas con refinaciones(RBR), utilizando los datos del vector anteriormente obtenidos.

Como lo explican en el paper se utiliza RBR con funcion de optimizacion I2, obteniendo como resultado el vector de clustering que se utiliza en el siguiente codig

In [13]:
import json
import glob
import operator

h = {}
files = (glob.glob("DataJson/*"))

files = sorted(files)

# Cluster array
cluster = [4,0,3,5,1,5,2,2,0,6,1,6,4,3,6,5]
#cluster = vectors.data

final = {}

def print_cluster( id, cluster ):
    print ("Cluster: ", id)

    row = ""
    counter = 0
    for item in cluster:
        row += item[0] + "\t"
        counter += 1
        if counter == 10:
            print (row)
            row = ""
            counter = 0
    print (row)
    print ("\n")

counter = 0
for file in files:
    h = json.load(open(file))
    sorted_h = sorted(h.items(), key=operator.itemgetter(1))
    x = sorted_h[-30:-1]
    for w,v in x:
        if cluster[counter] not in final:
            final[cluster[counter]] = {}
        else:
            if w not in final[cluster[counter]]:
                final[cluster[counter]][w] = 1
            else:
                final[cluster[counter]][w] += 1
    counter+=1



for h in final:
  print_cluster( h, sorted(final[h].items(), key=operator.itemgetter(1)) )



Cluster:  4
funcionar	hacer	huérfano	llorar	llover	maldecir	mi_o	nada	palabra	plantar	
responder	seco	sin	visible	alguno	decir	desilusionar	dios	el	ir	
pertenecer	suerte	vez	estar	mal	abandonar	romper	trigo	calentura	cama	
castigar	cuerpo	figura_o	hasta	miedo	para	saciar	se	sentar	solo	
ver	tener	bellaco	meter	te	él	tirar	seguir	que	perro	
me	le	castigo	querer	ser	


Cluster:  0
cariño	intención	jugar	me	nadie	siempre	sil_vado	silvando	sin	amigo	
caminar	culpa	fin	fracaso	mi	perder	recordar	triste	abandonar	sólo	
amor	estar	para	usar	vanesa	aquí	conseguir	decir	mujeriego	paciencia	
pensar	perfecto	pero	perro	sentar	sentir	ese	ir	nunca	solo	
tú	compañero	querer	tener	mami	contigo	caballero	con	saber	ser	
haber	por	


Cluster:  3
ver	vida	baba	crear	creer	cual	deber	este	idea	le	
mirar	nariz	parado	pensar	querer	quien	uno	vecino	verde	alrededor	
ellos	por	salir	todo	ser	lo	mientras	morir	de	el	
señor	abajo	afuera	alto	ancho	arriba	perdón	profundo	grande	maravilloso	
él	amor	estar	que	tan

Aqui se ve la separacion de los clusters obtenidos

Si lo desean pueden descargar el software y probar ustedes mismos con distintas letras de canciones

link software CLUTO: http://glaros.dtc.umn.edu/gkhome/cluto/cluto/overview

Ejercicio sacado de: https://github.com/datovard/LyricsClustering

**Comentarios:** En este taller se utilizan las letras de 16 canciones de direntes géneros musicales, que varian desde reggaeton a balada pop, se utiliza clustering para agrupar las palabras en 6 clusters.